# Fly Scanning

In this notebook you will:

* Learn how bluesky and ophyd implement a "fly scan" and the various methods involved.

We'll start by building a not-quite-realistic example to illustrate the process.

In [ ]:
from ophyd.sim import DeviceStatus


class DummyFlyer:
    def __init__(self, name):
        self.name = name
        self.parent = None

    def kickoff(self):
        status = DeviceStatus(self)
        
        # In a real example, we would set up something like this
        # to mark the status as finished asyncronously in response to
        # a change in some signal:
        #
        # def cb(value, old_value, **junk):
        #     if value == FOO and old_value == BAR:
        #         status._finished()
        # self.ready_signal.subscribe(cb)
        #
        # but in this simplistic example we'll just mark it finished right away:
        status._finished()
        
        return status
       
    def describe_collect(self):
        return {self.name: {'dtype': 'number', 'shape': [], 'source': 'SIM:...'}}

    def read_configuration(self):
        return OrderedDict()

    def describe_configuration(self):
        return OrderedDict()

    def complete(self):
        # same scheme as 'kickoff'...
        status = DeviceStatus(self)
        status._finished()
        return status

    def collect(self):
        # In a real example, we would read data and timestamps off of some
        # buffer on the hardware. In this case, we just simulate some data.
        for i in range(10):
            yield {'data': {'a': i**2},
                   'timestamps': {'a': i},
                   'time': i,
                   'seq_num': i}

    def stop(self, *, success=False):
        pass

In [ ]:
flyer = DummyFlyer('flyer')

First the RunEngine "kicks off" the flyer to tell it to begin capturing data. It immediately returns a status object and (later) marks that status object as 'done' to confirm that it has successfully begun acquiring.

In [ ]:
status = flyer.kickoff()
status.done

When we are ready to retrieve the data, we tell the flyer, "We're ready when you're ready." Sometimes the flyer is just waiting for us to tell it to stop, and so it will mark the status object as 'done' almost immediately (example: CSX's waveform collector). Other times, the flyer is performing some pre-planned routine, and it will make us wait until it finishes (example: HXN fly scans).

In [ ]:
status = flyer.complete()
status.done

Finally, we can collect the data off of the device and into Python.

In [ ]:
results = flyer.collect()
results

This is a generator. It yields the results one at a time, so we have the option of streaming them if the total dataset is too large for RAM.

In [ ]:
list(results)

Consider this [real example](https://github.com/NSLS-II-LIX/profile_collection/blob/master/startup/25-hplc.py).